In [ ]:
import yaml
import json
import os

path = os.getcwd() + '/topologies/second_subgraph/'
configs = os.listdir(path)

with open('data/resultData_fixed_ids.json','r') as f:
    data = json.load(f)

nodes = {}
for node in data['nodeList']:
    nodes[node['id']] = node

sources = [] # access
destinations = [] # kernel or mixed 

for config in configs:
    id = int(config.split('.')[0])

    if nodes[id]['deviceLevel'] == 'Access':
        sources.append(id)
    else:
        destinations.append(id) 

print("src:",len(sources), "\ndst:",len(destinations))

In [ ]:
SIMULATION_TIME = 600   # (seconds)
PAIRS_PER_SRC = {'mu': 3, 'sigma': 1}  # (seconds)
MSG_FREQUENCY = 2       # exponential dist (seconds)
MSG_SIZE = 20_000      # (bytes)
PACKET_SIZE = 1400      # (bytes)

In [ ]:
import random

connections = {}
for src in sources:
    num_pairs = int(random.normalvariate(**PAIRS_PER_SRC))
    while num_pairs <= 0:
        num_pairs = int(random.normalvariate(**PAIRS_PER_SRC))
        
    connections[src] = random.sample(sources, k=num_pairs)


In [218]:
'''
    Generation Logic:
        - For each source - while the current message time is less than the simulation time
            - pick the next message time (exp dist)
            - pick a random dest (uniform)
            - pick a random message (size)
            - split the message into packets based on packets size
'''

packets = []
unique_packet_id = 0
for src, pairs in connections.items():
    current_time = random.expovariate(1/MSG_FREQUENCY)
    while current_time <= SIMULATION_TIME:
        dst = random.choice(pairs)
        message_size = random.expovariate(1/MSG_SIZE)
        
        num_full_packets = int(message_size // PACKET_SIZE)
        last_packet_size = message_size % PACKET_SIZE

        for _ in range(num_full_packets):
            packets.append(
                {   
                    'unique_id': unique_packet_id,
                    'src': src,
                    'dst': dst,
                    'size': PACKET_SIZE,
                    'timestamp': current_time
                }
            )
            unique_packet_id += 1

        if last_packet_size > 0:
            packets.append(
                {
                    'unique_id': unique_packet_id,
                    'src': src,
                    'dst': dst,
                    'size': last_packet_size,
                    'timestamp': current_time
                }
            )
            unique_packet_id += 1

        # print("sum of packets:", num_full_packets * PACKET_SIZE + last_packet_size,
        #        "message size:", message_size,
        #        "EQ? =", num_full_packets * PACKET_SIZE + last_packet_size == message_size)
  
        current_time += random.expovariate(1/MSG_FREQUENCY)
    

In [226]:
with open("trace_second_SG.txt", 'w+') as f:
    for packet in packets:
        packet_info = f"{str(packet['unique_id'])} {str(packet['src'])} {str(packet['dst'])} {str(packet['size'])} {str(packet['timestamp'])}\n"
        f.write(packet_info)


KeyError: 'src'